In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

## Inisialisasi Data

In [95]:
m1 = float(input("Massa Pendulum 1: "))
m2 = float(input("Massa Pendulum 2: "))
L1 = float(input("Panjang Tali 1: "))
L2 = float(input("Panjang Tali 2: "))
teta1 = float(input("Sudut Pendulum 1: "))
teta2 = float(input("Sudut Pendulum 2: "))
dt = float(input("Step Size: "))
teta= np.array([[teta1*np.pi/180],[teta2*np.pi/180]])
tetadot = np.array([[0], [0]])
# print(m1,m2,L1,L2,teta1,teta2,dt,teta,tetadot)


## Kalkulasi Inersia

In [17]:
i1 = m1*(L1**2)
i2 = m2*(L2**2)
# print(i1,i2)

9.0 32.0


## Posisi Garis dan Pendulum

In [81]:
def pos():
    global L1, L2, teta
    pos_garis = np.array([[0, 0],
                          [L1*np.sin(teta[0, 0]), (-L1*np.cos(teta[0, 0]))],
                          [L1*np.sin(teta[0, 0])+L2*np.sin(teta[1, 0]), -(L1*np.cos(teta[0, 0])+L2*np.cos(teta[1, 0]))]])
    pos_massa = np.array([[1/2*L1*np.sin(teta[0, 0]), -1/2*L1*np.cos(teta[0, 0])], [L1*np.sin(
        teta[0, 0])+1/2*L2*np.sin(teta[1, 0]), -(L1*np.cos(teta[0, 0])+1/2*L2*np.cos(teta[1, 0]))]])
    # print(pos_garis)
    # print(pos_massa)
    return pos_garis, pos_massa


## Fungsi Pendulum

In [73]:
def inverse(m):
    determinant = m[0][0]*m[1][1]-m[0][1]*m[1][0]
    # special case for 2x2 matrix:
    return np.array([[m[1][1], -1*m[0][1]],
                     [-1*m[1][0], m[0][0]]])/determinant


def pendulum(teta,tetadot):
    global m1, m2, L1, L2, i1, i2
    g = 9.8
    tau = np.array([[0], [0]])
    M = np.array([[((m1/4)+m2)*(L1**2)+i1, m2*L1*L2*np.cos(teta[0,0]-teta[1,0])/2],
                 [m2*L1*L2*np.cos(teta[0,0]-teta[1,0])/2, m2*(L2**2)/4+i2]])
    # print("M")
    # print(M)
    Minv = inverse(M)
    # print("Minv")
    # print(Minv)
    matrix1 = np.dot(Minv, tau)
    # print(matrix1)
    # print(matrix1.shape)

    C = np.array([[m2*L1*L2*np.sin(teta[0,0]-teta[1,0])*tetadot[1,0]/2, -1*m2*L1*L2*np.sin(teta[0,0]-teta[1,0])*(tetadot[0,0]-tetadot[1,0])/2],
                 [-1*m2*L1*L2*np.sin(teta[0,0]-teta[1,0])*(tetadot[0,0]-tetadot[1,0])/2, -1*m2*L1*L2*np.sin(teta[0,0]-teta[1,0])*tetadot[0,0]/2]])
    # print("C")
    # print(C)
    A = np.dot(Minv, C)
    # print("A")
    # print(A)
    matrix2 = np.dot(A, tetadot)
    # print("matrix2")
    # print(matrix2)
    # print(matrix2.shape)

    G = np.array([[((m1/2)+m2)*g*L1*np.sin(teta[0,0])],
                  [m2*g*L2*np.sin(teta[1,0])/2]])
    # print("G")
    # print(G)
    matrix3 = np.dot(Minv, G)
    # print("matrix3")
    # print(matrix3)
    # print(matrix3.shape)

    return matrix1-matrix2-matrix3


## Fungsi Runge Kutta

In [74]:
def rungekuttaupdate(step):
    global teta, tetadot
    tetadotdot = pendulum(teta, tetadot)
    # print(tetadotdot)
    # print("k1")
    k1 = step/2*tetadotdot
    # print(k1)
    # print("k2")
    k2 = step/2*pendulum(teta+(step/2)*(tetadot+k1/2), tetadot+k1)
    # print(k2)
    # print("k3")
    k3 = step/2*pendulum(teta+(step/2)*(tetadot+k1/2), tetadot+k2)
    # print(k3)
    # print("k4")
    k4 = step/2*pendulum(teta+step*(tetadot+k3), tetadot+2*k3)
    # print(k4)
    teta = teta + step*(tetadot+(k1+k2+k3)/3)
    tetadot = tetadot + (k1+(2*k2)+(2*k3)+k4)/3


## Animasi

In [97]:
%matplotlib
fig = plt.figure(figsize=(10, 10), constrained_layout=True)
gs = fig.add_gridspec(4, 6)
ax_pendulum = fig.add_subplot(gs[:, :3])
ax_teta1 = fig.add_subplot(gs[0, 3:])
ax_teta2 = fig.add_subplot(gs[1, 3:])
ax_tetadot1 = fig.add_subplot(gs[2, 3:])
ax_tetadot2 = fig.add_subplot(gs[3, 3:])

pos_garis, pos_massa = pos()
m_plot, = ax_pendulum.plot(pos_massa[:, 0], pos_massa[:, 1], 'bo')
lin_plot, = ax_pendulum.plot(pos_garis[:, 0], pos_garis[:, 1], '-')


def init():
    global L1, L2
    ax_pendulum.set_xlim(-(L1+L2+0.1), (L1+L2+0.1))
    ax_pendulum.set_ylim(-(L1+L2+0.1), (L1+L2+0.1))
    return m_plot,

def plotanimation(step):
    rungekuttaupdate(step)
    pos_garis, pos_massa = pos()
    # print(pos_massa)
    # print("1")
    m_plot.set_data(pos_massa[:, 0], pos_massa[:, 1])
    lin_plot.set_data(pos_garis[:, 0], pos_garis[:, 1])
    return m_plot,lin_plot,

ani = FuncAnimation(fig, plotanimation, frames=np.arange(
    0, 2, dt), init_func=init, blit=True)

plt.show


Using matplotlib backend: Qt5Agg


<function matplotlib.pyplot.show(*, block=None)>